## Jupyter Notebook Module Reloads

In [18]:
# %load_ext autoreload
# %autoreload 2


In [19]:
!ipython kernel install --name "local-venv-kernel" --user

In [20]:
import configparser
import yaml
import os

# 🚨 Reverse Engineering Original Pipeline Dataflow Logic Start

In [21]:
# -*- coding: utf-8 -*-

### Import Relevant Libraries
import os

# ⬇️ RETURN TO THIS AFTER THE DATA FLOW LOGIC CONFIRMED ⬇️

In [22]:

# PYTHON_FOLDER = os.environ['PYTHON_FOLDER']  # Requires an environment variable to be preset
# os.chdir(PYTHON_FOLDER)


# ⬆️  RETURN TO THIS AFTER THE DATA FLOW LOGIC CONFIRMED ⬆️ 

In [23]:

import pandas as pd
import numpy as np
import configparser

import sys


from python import bc_transform as bct
# from python import bc_transform as bct
from python import bc_load_service_account as bcload

### Set up Python output to show every dataframe column
pd.set_option('display.max_columns', 500)


In [24]:



### Set variables by reading from the config.ini file
company_name = 'ServBiz'
# config = configparser.ConfigParser()
# config.read('config.yaml')

#GOOGLE_CREDENTIALS_FILE = config[company_name]['GOOGLE_CREDENTIALS_FILE']
GOOGLE_CREDENTIALS_FILE = '/Users/christophertrauco/skeleton/python/credentials.json'

# GOOGLE_SPREADSHEET_KEY = config[company_name]['GOOGLE_SPREADSHEET_KEY']
GOOGLE_SPREADSHEET_KEY = '1ETXNPRMjKcsbIdKAziQNuG1Llo_T5U3ayzO1ugiW9_o'


# RAW_DATAFILE = config[company_name]['RAW_DATAFILE']
RAW_DATAFILE = 'https://raw.githubusercontent.com/theventurecity/data-toolkit/master/data/ServBiz_transactions.csv'

### Extract raw data
t = pd.read_csv(RAW_DATAFILE)

### Instantiate TVCLoad object with Google credentials file to write to Google Sheets
bcl = bcload.BCLoad(GOOGLE_CREDENTIALS_FILE)


### Define segments. Each Segment name maps to a segment_col name
segments = {'Unsegmented' : None,
            'Agent - ' : 'segment'
            }


for seg in segments:
    print('Processing the', seg, 'segment')
    
    seg_col = segments[seg]
    if seg_col is None:
        use_seg = False
    else:
        use_seg = True

    

In [25]:

    # ### Transform the raw data into dau_decorated
    # dau = bct.create_dau_df(t, 
    #                          user_id = 'client_id', 
    #                          activity_date = 'date', 
    #                          inc_amt = 'value_usd',
    #                          segment_col = seg_col
    #                         )
    
    # dau_decorated = bct.create_dau_decorated_df(dau)
    
    
    # ### Calculate Weekly Growth Accounting and Cohort Analysis based on wau_decorated
    # # WAU Decorated
    # wau_decorated = bct.create_xau_decorated_df(dau_decorated, 'week', use_segment=use_seg)


In [26]:
# from clean_file import clean_file

In [27]:
from python import bc_transform

## Output No. 1 -  All Agent Intents

In [31]:
from python import bc_transform
df = pd.DataFrame(bc_transform.Intents.create_intents_df())

bcl.write_to_google_sheet(df, seg + 'Intents', GOOGLE_SPREADSHEET_KEY)

## Output No. 2 - All Agent Entities

In [33]:
# Output of all agent entity type values and mappings
bc_transform.EntityTypes.list_entity_types()
# bc_transform.EntityTypes.clean_data()
df2 = pd.DataFrame(bc_transform.EntityTypes.create_df())

bcl.write_to_google_sheet(df2, seg + 'entityTypes', GOOGLE_SPREADSHEET_KEY)

ModuleNotFoundError: No module named 'clean_file'

In [ ]:

    # Weekly Growth Accounting
    w_ga = bct.consolidate_all_ga(wau_decorated, 'week', 
                                     use_segment = use_seg, 
                                     growth_rate_periods = 12, 
                                     keep_last_period = False)
    bcl.write_to_google_sheet(w_ga, seg + ' Weekly Growth Accounting', GOOGLE_SPREADSHEET_KEY)
    
    # Weekly Cohorts
    wau_cohorts = bct.create_xau_cohort_df(wau_decorated, 'week', use_segment = use_seg)
    bcl.write_to_google_sheet(wau_cohorts, seg + ' Weekly Cohorts', GOOGLE_SPREADSHEET_KEY)
    
    
    
    ### Calculate Monthly Growth Accounting and Cohort Analysis based on mau_decorated
    # MAU Decorated
    mau_decorated = bct.create_xau_decorated_df(dau_decorated, 'month', use_segment=use_seg)
    
    # Monthly Growth Accounting
    m_ga = bct.consolidate_all_ga(mau_decorated, 'month', 
                                     use_segment = use_seg, 
                                     growth_rate_periods = 12, 
                                     keep_last_period = False)
    tvcl.write_to_google_sheet(m_ga, seg + ' Monthly Growth Accounting', GOOGLE_SPREADSHEET_KEY)
    
    # Monthly Cohorts
    mau_cohorts = bct.create_xau_cohort_df(mau_decorated, 'month', use_segment=use_seg)
    bcl.write_to_google_sheet(mau_cohorts, seg + ' Monthly Cohorts', GOOGLE_SPREADSHEET_KEY)
    
    
    
    ### Calculate the Rolling 28-Day DAU/MAU ratios
    rolling_dau_mau = bct.create_xau_window_df(dau_decorated, 
                                              time_period = 'day',
                                              window_days = 28, 
                                              breakouts = [2, 4, 8, 12, 16, 20],
                                              use_segment = use_seg,
                                              use_final_day = False)
    bcl.write_to_google_sheet(rolling_dau_mau, seg + ' Rolling DAU/MAU', GOOGLE_SPREADSHEET_KEY)
    
    
    ### Calculate the Rolling 28-Day WAU/MAU ratios
    rolling_wau_mau = bct.create_xau_window_df(dau_decorated, 
                                              time_period = 'week',
                                              window_days = 28, 
                                              breakouts = [2, 3, 4],
                                              use_segment = use_seg,
                                              use_final_day = False)
    bcl.write_to_google_sheet(rolling_wau_mau, seg + ' Rolling WAU/MAU', GOOGLE_SPREADSHEET_KEY)